In [1]:
import openrouteservice as ors
import folium
import math

In [2]:
# Chave da API
client = ors.Client(key='5b3ce3597851110001cf6248db9786ac4c7446dea93a21d0c244d5b2')

# Gera o mapa da região
mapa = folium.Map(location=list(reversed([-47.976167,-15.922460])), tiles="cartodbpositron", zoom_start = 11)
mapa

In [3]:
# Coordenadas da rota
coordenadas = [[-48.045285,-15.989108], [-47.882799,-15.791886]]

# Define o perfil da rota
rota = client.directions(coordinates = coordenadas, profile = 'driving-car', format = 'geojson')

# Traça a rota no mapa
folium.PolyLine(locations=[list(reversed(coord)) for coord in rota['features'][0]['geometry']['coordinates']], color='blue').add_to(mapa)


mapa

In [4]:
# Marca o local de inicio
rodoviaria = [-47.8833709,-15.7934468]
mapa = folium.Map(location=list(reversed([-47.885729,-15.798224])), tiles="cartodbpositron", zoom_start = 13)
folium.Marker(location=list(reversed(rodoviaria)), icon=folium.Icon(color="red")).add_to(mapa)

# Divide as coordenadas e cria rotas para três veiculos
coords = [
    [-47.894583,-15.810909],
    [-47.962913,-15.896979],
    [-47.990700,-15.945459],
    [-48.047356,-15.989687],
    [-47.874270,-15.775838],
    [-47.866248,-15.770699],
    [-47.869608,-15.757577],
    [-47.876013,-15.753971],
    [-47.890927,-15.795877],
    [-47.926040,-15.759948],
    [-47.932124,-15.788361],
]

vehicles = [
    ors.optimization.Vehicle(id=0, profile='driving-car', start=rodoviaria, end=rodoviaria, capacity=[4]),
    ors.optimization.Vehicle(id=1, profile='driving-car', start=rodoviaria, end=rodoviaria, capacity=[4]),
    ors.optimization.Vehicle(id=2, profile='driving-car', start=rodoviaria, end=rodoviaria, capacity=[3]),
]

jobs = [ors.optimization.Job(id=index, location=coords, amount=[1]) for index, coords in enumerate(coords)]
optimized = client.optimization(jobs=jobs, vehicles=vehicles, geometry=True)
line_colors = ['green', 'blue', 'yellow']
for route in optimized['routes']:
    folium.PolyLine(locations=[list(reversed(coords)) for coords in ors.convert.decode_polyline(route['geometry'])['coordinates']], color=line_colors[route['vehicle']]).add_to(mapa)

mapa

In [5]:
# Organiza a rota de acordo com o tempo
coords = [
  { 'location': [-47.894583,-15.810909],'service': 1*60*60 },
  { 'location': [-47.962913,-15.896979],'service': 0.5*60*60 },
  { 'location': [-47.990700,-15.945459],'service': 0.75*60*60 },
  { 'location': [-48.047356,-15.989687],'service': 1*60*60 },
  { 'location': [-47.874270,-15.775838],'service': 0.25*60*60 },
  { 'location': [-47.866248,-15.770699],'service': 0.3*60*60 },
  { 'location': [-47.869608,-15.757577],'service': 0.2*60*60 },
  { 'location': [-47.876013,-15.753971],'service': 0.1*60*60 },
]

rodoviaria = [-47.8833709,-15.7934468]
mapa = folium.Map(location=list(reversed([-47.885729,-15.798224])), tiles="cartodbpositron", zoom_start = 13)
folium.Marker(location=list(reversed(rodoviaria)), icon=folium.Icon(color="red")).add_to(mapa)

vehicles = [
    ors.optimization.Vehicle(id=0, profile='driving-car', start=rodoviaria, end=rodoviaria, time_window=[0, 2*60*60]),
    ors.optimization.Vehicle(id=1, profile='driving-car', start=rodoviaria, end=rodoviaria, time_window=[0, 4*60*60])
]
jobs = [ors.optimization.Job(id=index, **job) for index, job in enumerate(coords)]
optimized = client.optimization(jobs=jobs, vehicles=vehicles, geometry=True)
line_colors = ['green', 'orange', 'blue', 'yellow']
for route in optimized['routes']:
    folium.PolyLine(locations=[list(reversed(coords)) for coords in ors.convert.decode_polyline(route['geometry'])['coordinates']], color=line_colors[route['vehicle']]).add_to(mapa)
    for step in route['steps']:
        if not step['type'] == 'job':
            continue
        folium.Marker(location=list(reversed(step['location'])), popup=f"Arrival time: {math.floor(step['arrival'] / (60*60))} hours {math.floor((step['arrival'] % (60*60)) / 60)} minutes", icon=folium.Icon(color=line_colors[route['vehicle']])).add_to(mapa)

mapa